In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'geopandas'

do based of regions 2dmap of us

Regions:
- California
- West (WA, OR, ID, MT, WY, NV, UT, CO, AZ, NM)
- Plains (ND, SD, MN, NE, IA, KS, MO)
- Midsouth (KY, VA, TN, MD, NC, SC)
- Northeast (WV, MD, DE, NJ, CN, RI, MA, VT, NH, NY, PA, ME)
- Southeast (MS, AL, GA, FL)
- South Central (OK, AR, TX, LA)
- Great Lakes (WI, IL, IN, MI, OH)

In [ ]:
# Load a GeoDataFrame with the USA administrative areas shapefile
gdf_usa = gpd.read_file('data/cb_2018_us_state_500k.zip')
print(gdf_usa.columns)
gdf_usa.head(5)


In [ ]:
hasdata_df = pd.read_csv("data/hasstotalmetrics.csv")
print(hasdata_df.columns)
hasdata_df.head(10)

In [ ]:
# Now, sum up '2023Units' for each region across all periods
total_volume_by_region = hasdata_df.groupby('Geography')['Total2023Units'].sum().reset_index()
# Display the aggregated data to check it
print(total_volume_by_region)

In [ ]:
import geopandas as gpd
import plotly.express as px

# Load the state-level GeoDataFrame
gdf_states = gpd.read_file('data/cb_2018_us_state_500k.zip')

# Define your state to region mapping correctly
state_to_region = {
    'California': 'California',
    'Washington': 'West', 'Oregon': 'West', 'Idaho': 'West', 'Montana': 'West', 'Wyoming': 'West',
    'Nevada': 'West', 'Utah': 'West', 'Colorado': 'West', 'Arizona': 'West', 'New Mexico': 'West',
    'North Dakota': 'Plains', 'South Dakota': 'Plains', 'Minnesota': 'Plains', 'Nebraska': 'Plains',
    'Iowa': 'Plains', 'Kansas': 'Plains', 'Missouri': 'Plains',
    'Kentucky': 'Midsouth', 'Virginia': 'Midsouth', 'Tennessee': 'Midsouth', 'Maryland': 'Midsouth',
    'North Carolina': 'Midsouth', 'South Carolina': 'Midsouth',
    'West Virginia': 'Northeast', 'Delaware': 'Northeast', 'New Jersey': 'Northeast', 'New York': 'Northeast',
    'Connecticut': 'Northeast', 'Rhode Island': 'Northeast', 'Massachusetts': 'Northeast', 'Vermont': 'Northeast',
    'New Hampshire': 'Northeast', 'Pennsylvania': 'Northeast', 'Maine': 'Northeast',
    'Mississippi': 'Southeast', 'Alabama': 'Southeast', 'Georgia': 'Southeast', 'Florida': 'Southeast',
    'Oklahoma': 'South Central', 'Arkansas': 'South Central', 'Texas': 'South Central', 'Louisiana': 'South Central',
    'Wisconsin': 'Great Lakes', 'Illinois': 'Great Lakes', 'Indiana': 'Great Lakes', 'Ohio': 'Great Lakes', 'Michigan': 'Great Lakes'
}

# Map each state in the GeoDataFrame to a region
gdf_states['Region'] = gdf_states['NAME'].apply(lambda x: state_to_region.get(x))

# Dissolve states into regions
gdf_regions = gdf_states.dissolve(by='Region', aggfunc='sum').reset_index()

# Ensure your avocado data (hasdata_df) is loaded correctly and structured properly
# Merge the GeoDataFrame with your avocado sales data
merged_gdf = gdf_regions.merge(hasdata_df, left_on='Region', right_on='Geography', how='left')

# Replace NaN values with 0 for regions that do not have matching avocado data
merged_gdf['Total2023Units'].fillna(0, inplace=True)

# Ensure the geometry is in a format that Plotly can use
geojson = merged_gdf.geometry.__geo_interface__

# Generate the choropleth map using Plotly
fig = px.choropleth(merged_gdf,
                    geojson=geojson,
                    locations=merged_gdf.index,
                    color='Total2023Units',
                    color_continuous_scale='Viridis',
                    scope="usa",
                    labels={'Total2023Units': 'Total Avocado Volume 2023'})

# Enhance the layout
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations")

# Show the figure
fig.show()